# DATA CLEANING


In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller


# Read the Excel file
df = pd.read_excel('GovtBonds+FPIs.xlsx')

# Create a DataFrame with Date, YLD_YTM_MID, and Total columns
yield_total_df = df[['Date', 'YLD_YTM_MID', 'Total']].copy()

# Display the new DataFrame
print(yield_total_df)



          Date  YLD_YTM_MID  Total
0   2011-01-31        8.545   5366
1   2011-02-28        8.517  -3268
2   2011-03-31        8.349   6880
3   2011-04-29        8.475   7195
4   2011-05-31        8.668  -4275
..         ...          ...    ...
130 2024-05-31        7.224 -12911
131 2024-06-28        7.035  41757
132 2024-07-31        6.997  48796
133 2024-08-30        6.940  25493
134        NaT        6.853  93538

[135 rows x 3 columns]


In [3]:
yield_total_df['Yld_Returns'] = np.log(yield_total_df['YLD_YTM_MID'] / yield_total_df['YLD_YTM_MID'].shift(1))
yield_total_df['Total_Returns'] = np.log(yield_total_df['Total'] / yield_total_df['Total'].shift(1))

# Drop rows with NaN values resulting from the shift operation
log_returns_df = yield_total_df[['Date', 'Yld_Returns','Total_Returns']].dropna()

# Display the DataFrame with log returns
print(log_returns_df)

          Date  Yld_Returns  Total_Returns
3   2011-04-29     0.014979       0.044768
6   2011-07-29     0.020278       0.779672
8   2011-09-30     0.000116      -1.444108
12  2012-01-31    -0.018621       0.185175
13  2012-02-29    -0.005701       0.291294
..         ...          ...            ...
127 2024-02-29    -0.026945      -0.977189
128 2024-03-29    -0.002384       0.491166
130 2024-05-31    -0.003593      -0.230628
132 2024-07-31    -0.005416       0.155781
133 2024-08-30    -0.008180      -0.649244

[83 rows x 3 columns]


e:\Rouhin Projects\Nityoday. Geetha. I RP\.venv\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [4]:
result = adfuller(yield_total_df['Total_Returns'].dropna())

# Print the test results
print('ADF Statistic:', result[0])
print('p-value:', result[1])
print('Critical Values:')
for key, value in result[4].items():
    print(f'   {key}: {value}')

# Check if the series is stationary
if result[1] < 0.05:
    print("The Total_return series is stationary.")
else:
    print("The series is not stationary.")

result_yld = adfuller(yield_total_df['Yld_Returns'].dropna())

# Print the test results for Yld_Returns
print('ADF Statistic for Yld_Returns:', result_yld[0])
print('p-value for Yld_Returns:', result_yld[1])
print('Critical Values for Yld_Returns:')
for key, value in result_yld[4].items():
    print(f'   {key}: {value}')

# Check if the Yld_Returns series is stationary
if result_yld[1] < 0.05:
    print("The Yld_Returns series is stationary.")
else:
    print("The Yld_Returns series is not stationary.")

ADF Statistic: -10.300139435094742
p-value: 3.3890961216567764e-18
Critical Values:
   1%: -3.512738056978279
   5%: -2.8974898650628984
   10%: -2.585948732897085
The Total_return series is stationary.
ADF Statistic for Yld_Returns: -10.319669832379766
p-value for Yld_Returns: 3.0321692318249343e-18
Critical Values for Yld_Returns:
   1%: -3.4808880719210005
   5%: -2.8836966192225284
   10%: -2.5785857598714417
The Yld_Returns series is stationary.


In [5]:
# Generate descriptive statistics
descriptive_stats = yield_total_df.describe()

# Display the descriptive statistics
print(descriptive_stats)

                                Date  YLD_YTM_MID          Total  Yld_Returns  \
count                            134   135.000000     135.000000   134.000000   
mean   2017-11-10 20:46:34.029850624     7.672267    8565.600000    -0.001647   
min              2011-01-31 00:00:00     6.490000 -118203.000000    -0.083828   
25%              2014-08-15 06:00:00     7.166500   -4586.500000    -0.019345   
50%              2017-09-30 12:00:00     7.562000   10653.000000    -0.002300   
75%              2021-04-22 12:00:00     8.163500   22555.000000     0.014577   
max              2024-08-30 00:00:00     9.420000   93538.000000     0.102474   
std                              NaN     0.664613   26186.145217     0.027643   

       Total_Returns  
count      84.000000  
mean       -0.028975  
min        -2.977365  
25%        -0.653546  
50%        -0.037258  
75%         0.492238  
max         3.356071  
std         1.123496  


In [6]:
# Perform Augmented Dickey-Fuller test
def adf_test(series, title=''):
    """
    Pass in a time series and an optional title, returns an ADF report
    """
    print(f'Augmented Dickey-Fuller Test: {title}')
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Test Statistic', 'p-value', '# Lags Used', '# Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    print(out.to_string())
    print('')

# Perform ADF test on Yld_Returns and Total_Returns
adf_test(log_returns_df['Yld_Returns'], title='Yld_Returns')
adf_test(log_returns_df['Total_Returns'], title='Total_Returns')

Augmented Dickey-Fuller Test: Yld_Returns
ADF Test Statistic     -8.542938e+00
p-value                 9.672782e-14
# Lags Used             1.000000e+00
# Observations Used     8.100000e+01
Critical Value (1%)    -3.513790e+00
Critical Value (5%)    -2.897943e+00
Critical Value (10%)   -2.586191e+00

Augmented Dickey-Fuller Test: Total_Returns
ADF Test Statistic     -1.028071e+01
p-value                 3.786235e-18
# Lags Used             1.000000e+00
# Observations Used     8.100000e+01
Critical Value (1%)    -3.513790e+00
Critical Value (5%)    -2.897943e+00
Critical Value (10%)   -2.586191e+00



In [7]:
# ADF test with intercept
result_intercept = adfuller(log_returns_df['Total_Returns'], regression='c')
print('ADF Test with Intercept:')
print('ADF Statistic:', result_intercept[0])
print('p-value:', result_intercept[1])
print('Critical Values:')
for key, value in result_intercept[4].items():
    print(f'   {key}: {value}')
print('')

# ADF test with trend and intercept
result_trend_intercept = adfuller(log_returns_df['Total_Returns'], regression='ct')
print('ADF Test with Trend and Intercept:')
print('ADF Statistic:', result_trend_intercept[0])
print('p-value:', result_trend_intercept[1])
print('Critical Values:')
for key, value in result_trend_intercept[4].items():
    print(f'   {key}: {value}')
print('')

# ADF test with none
result_none = adfuller(log_returns_df['Total_Returns'], regression='n')
print('ADF Test with None:')
print('ADF Statistic:', result_none[0])
print('p-value:', result_none[1])
print('Critical Values:')
for key, value in result_none[4].items():
    print(f'   {key}: {value}')
print('')


ADF Test with Intercept:
ADF Statistic: -10.280709047906381
p-value: 3.786234609647783e-18
Critical Values:
   1%: -3.5137900174243235
   5%: -2.8979433868293945
   10%: -2.5861907285474777

ADF Test with Trend and Intercept:
ADF Statistic: -10.38363554330066
p-value: 3.666370251028592e-16
Critical Values:
   1%: -4.075121979053178
   5%: -3.4661550774027594
   10%: -3.1596113315495042

ADF Test with None:
ADF Statistic: -10.28613426339971
p-value: 2.3681082683659117e-18
Critical Values:
   1%: -2.593895281207133
   5%: -1.9447701769340342
   10%: -1.6139069172683327



In [8]:
# Calculate the correlation between Yld_Returns and Total_Returns
correlation = log_returns_df[['Yld_Returns', 'Total_Returns']].corr()

# Display the correlation matrix
print(correlation)

               Yld_Returns  Total_Returns
Yld_Returns       1.000000      -0.215887
Total_Returns    -0.215887       1.000000


In [9]:
from statsmodels.tsa.stattools import grangercausalitytests

# Perform Granger causality test
max_lag = 5
granger_test_result = grangercausalitytests(log_returns_df[['Yld_Returns', 'Total_Returns']], max_lag, verbose=True)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0003  , p=0.9853  , df_denom=79, df_num=1
ssr based chi2 test:   chi2=0.0004  , p=0.9850  , df=1
likelihood ratio test: chi2=0.0004  , p=0.9850  , df=1
parameter F test:         F=0.0003  , p=0.9853  , df_denom=79, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.0580  , p=0.9437  , df_denom=76, df_num=2
ssr based chi2 test:   chi2=0.1237  , p=0.9400  , df=2
likelihood ratio test: chi2=0.1236  , p=0.9401  , df=2
parameter F test:         F=0.0580  , p=0.9437  , df_denom=76, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.3310  , p=0.8030  , df_denom=73, df_num=3
ssr based chi2 test:   chi2=1.0882  , p=0.7799  , df=3
likelihood ratio test: chi2=1.0808  , p=0.7817  , df=3
parameter F test:         F=0.3310  , p=0.8030  , df_denom=73, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.3838  , p=0.8195  , df_d

e:\Rouhin Projects\Nityoday. Geetha. I RP\.venv\Lib\site-packages\statsmodels\tsa\stattools.py:1556: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


In [14]:
from statsmodels.tsa.api import VAR

# Select the columns for the VAR model
data_for_var = log_returns_df[['Yld_Returns', 'Total_Returns']]

# Fit the VAR model
model = VAR(data_for_var)
fitted_model = model.fit(maxlags=max_lag)

# Print the parameters of the fitted model
print(fitted_model.params)
# Print the summary of the fitted model
print(fitted_model.summary())

                  Yld_Returns  Total_Returns
const               -0.003606      -0.047129
L1.Yld_Returns      -0.125105       3.042345
L1.Total_Returns     0.001377      -0.341983
L2.Yld_Returns      -0.184070       2.401466
L2.Total_Returns     0.001896      -0.383369
L3.Yld_Returns       0.137265       1.397363
L3.Total_Returns     0.001258      -0.060793
L4.Yld_Returns       0.029998       0.353730
L4.Total_Returns     0.000128      -0.001378
L5.Yld_Returns      -0.005389       0.992227
L5.Total_Returns    -0.006452       0.045202
  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Wed, 20, Nov, 2024
Time:                     10:56:41
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                   -6.11313
Nobs:                     78.0000    HQIC:                  -6.51175
Log likelihood:           64.9816    FPE:                 0.00114304
AIC:     

e:\Rouhin Projects\Nityoday. Geetha. I RP\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
